# ДЗ 8: Fine-tuning + LangChain Tools

**Трек C — Инструктивный помощник**

1. Fine-tuning на FineTome-100k (следование инструкциям)
2. LangChain Tools: text_formatter, template_generator, structure_analyzer, content_validator
3. Интеграция: fine-tuned модель + tools, сравнение до/после, связка модель+tools

## 0. Установка зависимостей

> ⚠️ В Colab после `pip install` может понадобиться **Runtime → Restart session**.

In [ ]:
# Порядок установки: transformers + huggingface_hub должны быть совместимы (is_offline_mode)
# langchain-huggingface НЕ ставим — он требует hf_hub<1.0, а transformers 5.x нужен >=1.3
!pip install -q -U huggingface_hub>=1.3.0 transformers datasets peft accelerate bitsandbytes trl
!pip install -q langchain langchain-core langchain-community

## 1. Fine-tuning (FineTome-100k + LoRA)

**Выбор модели:** `tiny` — быстрый показ (~1 ч, публичная), `mistral` — качественнее (~5 ч).

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# tiny = ~1 ч (Colab T4, публичная), mistral = ~5 ч, качественнее
MODEL_MODE = "tiny"  # "tiny" | "mistral"

MODELS = {
    "tiny": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",   # 1.1B, публичная, быстрый
    "mistral": "mistralai/Mistral-7B-v0.1",
}
model_id = MODELS[MODEL_MODE]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print(f"Модель: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Подготовка FineTome-100k → instruction format
from datasets import load_dataset

ds = load_dataset("mlabonne/FineTome-100k", split="train")
# Размер зависит от модели: smol — 800 (~1 ч), mistral — 1500 (~5 ч)
N_SAMPLES = 800 if MODEL_MODE == "tiny" else 1500
ds = ds.select(range(N_SAMPLES))

def format_conversation(sample):
    conv = sample["conversations"]
    if len(conv) < 2:
        return None
    inst = next((m["value"] for m in conv if m.get("from") == "human"), "")
    resp = next((m["value"] for m in conv if m.get("from") == "gpt"), "")
    if not inst or not resp:
        return {"text": ""}
    text = f"""### Instruction:
{inst}

### Response:
{resp}"""
    return {"text": text}

ds = ds.map(format_conversation, remove_columns=ds.column_names)
ds = ds.filter(lambda x: x["text"] and len(x["text"]) > 10)
print(ds.num_rows, "примеров")

In [ ]:
from trl import SFTTrainer, SFTConfig

# T4: bf16 недоступен, fp16 даёт BFloat16/scaler конфликт → fp32 без AMP
try:
    cap = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
    use_bf16 = cap[0] >= 8
except Exception:
    use_bf16 = False
# На T4 fp16 даёт BFloat16/scaler конфликт → отключаем AMP
use_amp = use_bf16
print("Precision: fp32" if not use_amp else f"Precision: {'bf16' if use_bf16 else 'fp16'}")

trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    processing_class=tokenizer,
    args=SFTConfig(
        max_length=512,
        per_device_train_batch_size=2 if not use_amp else (8 if MODEL_MODE == "tiny" else 4),
        gradient_accumulation_steps=8 if not use_amp else 4,
        warmup_steps=15,
        max_steps=80 if MODEL_MODE == "tiny" else 120,
        learning_rate=2e-4,
        bf16=use_bf16 if use_amp else False,
        fp16=(not use_bf16 and use_amp),
        logging_steps=5,
        logging_first_step=True,
        logging_strategy="steps",
        report_to="none",
        output_dir="outputs_finetome",
        optim="paged_adamw_8bit",
        save_steps=40,
        save_total_limit=2,
        run_name="finetome_lora"
    ),
)

n_steps = 80 if MODEL_MODE == "tiny" else 120
eta = "~1 ч" if MODEL_MODE == "tiny" else "~4–5 ч"
print(f"Датасет: {len(ds)} примеров | Шагов: {n_steps} | {eta}")
print("Запуск обучения...")
result = trainer.train()
print(f"\nГотово. Loss: {result.training_loss:.4f} | Время: {result.metrics.get('train_runtime', 0):.0f} сек")
trainer.save_model("outputs_finetome/final")
tokenizer.save_pretrained("outputs_finetome/final")
print("Модель сохранена в outputs_finetome/final")

## 2. LangChain Tools

In [ ]:
from langchain_core.tools import tool


@tool
def text_formatter(text: str, width: int = 80, indent: int = 2) -> str:
    """Форматирует текст: перенос по width, отступ indent. Вход: text (строка), width (макс. символов в строке), indent (отступ)."""
    lines = text.replace("\n", " ").split()
    result = []
    current = ""
    prefix = " " * indent
    for w in lines:
        if len(current) + len(w) + 1 <= width:
            current = f"{current} {w}".strip() if current else w
        else:
            if current:
                result.append(prefix + current)
            current = w
    if current:
        result.append(prefix + current)
    return "\n".join(result) if result else text


@tool
def template_generator(template_type: str, placeholders: str = "") -> str:
    """Генерирует шаблон по типу. template_type: 'email'|'json'|'markdown'|'prompt'. placeholders — список полей через запятую."""
    templates = {
        "email": "Subject: {subject}\n\nDear {name},\n\n{body}\n\nBest regards,\n{sender}",
        "json": "{\"key\": \"value\"}",
        "markdown": "# {title}\n\n## Введение\n{intro}\n\n## Основная часть\n{content}\n\n## Заключение\n{conclusion}",
        "prompt": "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    }
    return templates.get(template_type.lower(), templates["prompt"])


@tool
def structure_analyzer(text: str) -> str:
    """Анализирует структуру текста: заголовки (#, ##), списки (-, *, 1.), параграфы. Возвращает описание структуры."""
    import json
    desc = {"headers": [], "lists": 0, "paragraphs": 0, "length_chars": len(text)}
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    for line in lines:
        if line.startswith("#"):
            level = len(line) - len(line.lstrip("#"))
            desc["headers"].append({"level": level, "text": line.lstrip("# ")[:50]})
        elif line.startswith(("-", "*", "1.", "2.")) or (len(line) > 1 and line[0].isdigit() and line[1] in ".)"):
            desc["lists"] = desc.get("lists", 0) + 1
        elif len(line) > 20:
            desc["paragraphs"] += 1
    return json.dumps(desc, ensure_ascii=False)


@tool
def content_validator(text: str, rules: str = "structure") -> str:
    """Проверяет текст по правилам. rules: 'structure' (есть заголовки/параграфы) | 'length' (не пустой) | 'format' (markdown/email). Возвращает JSON: valid (bool), issues (list)."""
    import json
    issues = []
    if rules == "length" or "length" in rules:
        if not text or not text.strip():
            issues.append("Текст пустой")
    if rules == "structure" or "structure" in rules:
        if "#" not in text and "\n\n" not in text:
            issues.append("Нет чёткой структуры (заголовки/параграфы)")
    if rules == "format" or "format" in rules:
        if "@" in text and "Subject:" not in text:
            issues.append("Похоже на email, но нет Subject")
    return json.dumps({"valid": len(issues) == 0, "issues": issues}, ensure_ascii=False)

In [ ]:
tools = [text_formatter, template_generator, structure_analyzer, content_validator]
for t in tools:
    print(t.name, ":", t.description[:60] + "...")

### 3.1 Сравнение до/после fine-tuning

Запускаем одинаковые промпты на базовой модели и на fine-tuned, чтобы оценить влияние обучения на следование инструкциям.

In [ ]:
import gc

# Тестовые промпты для сравнения
COMPARE_PROMPTS = [
    "Объясни, что такое рекурсия в программировании. Приведи короткий пример на Python.",
    "Напиши короткое приветственное письмо коллеге перед отпуском.",
]

def make_prompt(instruction):
    return f"""### Instruction:
{instruction}

### Response:
"""

def run_model(model, tokenizer, prompts, max_new=150):
    results = []
    for p in prompts:
        full = make_prompt(p)
        inp = tokenizer(full, return_tensors="pt").to(model.device)
        out = model.generate(**inp, max_new_tokens=max_new, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
        raw = tokenizer.decode(out[0][inp.input_ids.shape[1]:], skip_special_tokens=True)
        for stop in ["\n\n### Instruction", "### Instruction", "\n### Response"]:
            if stop in raw:
                raw = raw.split(stop)[0]
        results.append(raw.strip())
    return results

# 1) Базовая модель (без fine-tuning)
from transformers import BitsAndBytesConfig
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
base_tok = AutoTokenizer.from_pretrained(model_id)
base_tok.pad_token = base_tok.eos_token
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb, device_map="auto")
base_results = run_model(base_model, base_tok, COMPARE_PROMPTS)
print("=== ДО fine-tuning (базовая модель) ===")
for i, (p, r) in enumerate(zip(COMPARE_PROMPTS, base_results)):
    print(f"\n[{i+1}] Промпт: {p[:50]}...")
    print(f"Ответ: {r[:300]}...")
del base_model
gc.collect()
torch.cuda.empty_cache()

## 3. Интеграция: fine-tuned модель + tools

In [ ]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
from peft import AutoPeftModelForCausalLM

# Загрузка fine-tuned адаптера (или базовой модели, если обучение ещё не запущено)
adapter_path = "outputs_finetome/final"
model_id = MODELS[MODEL_MODE]

try:
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    tokenizer.pad_token = tokenizer.eos_token
    bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
    model = AutoPeftModelForCausalLM.from_pretrained(
        adapter_path,
        quantization_config=bnb,
        device_map="auto",
    )
except Exception:
    # Fallback: базовая модель (если адаптер не сохранён)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb, device_map="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# 2) Fine-tuned модель — те же промпты
finetuned_results = run_model(model, tokenizer, COMPARE_PROMPTS)
print("=== ПОСЛЕ fine-tuning ===")
for i, (p, r) in enumerate(zip(COMPARE_PROMPTS, finetuned_results)):
    print(f"\n[{i+1}] Промпт: {p[:50]}...")
    print(f"Ответ: {r[:300]}...")

In [ ]:
# Mistral не поддерживает tool calling нативно — используем прямые вызовы tools
# ReAct-агент доступен, но Mistral без tool-calling может давать нестабильный вывод.
# Используем прямые вызовы tools (см. ячейку ниже) как основной демо-сценарий.

In [ ]:
# Демо: прямой вызов модели (без tools)
def trim_response(text):
    """Обрезаем, если модель начала новый блок Instruction."""
    for stop in ["\n\n### Instruction", "### Instruction", "\n### Response"]:
        if stop in text:
            text = text.split(stop)[0]
    return text.strip()

def generate(prompt_text, max_new_tokens=200):
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    raw = tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return trim_response(raw)

user_prompt = "Объясни, что такое рекурсия в программировании. Приведи короткий пример на Python."
full_prompt = f"""### Instruction:
{user_prompt}

### Response:
"""

print("Ответ модели:")
print(generate(full_prompt))

In [ ]:
# Демо: прямой вызов tools
sample = "Длинный текст который нужно отформатировать по ширине 40 символов с отступом 2 пробела для читаемости."
print("text_formatter:", text_formatter.invoke({"text": sample, "width": 40, "indent": 2}))
print()
print("template_generator:", template_generator.invoke({"template_type": "prompt"}))
print()
print("structure_analyzer:", structure_analyzer.invoke({"text": "# Заголовок\n\n- пункт 1\n- пункт 2\n\nПараграф текста."}))
print()
print("content_validator:", content_validator.invoke({"text": "# Заголовок\n\nПараграф.", "rules": "structure"}))

### 3.2 Связка модель + tools

Сценарий: пользователь просит отформатировать текст → вызываем `text_formatter` → модель генерирует краткий ответ о выполненной работе.

In [ ]:
# Сценарий: запрос на форматирование → tool → модель подводит итог
user_request = "Отформатируй этот текст по ширине 50: Python это язык программирования высокого уровня. Он поддерживает объектно ориентированное программирование и имеет чистый синтаксис."
raw_text = "Python это язык программирования высокого уровня. Он поддерживает объектно ориентированное программирование и имеет чистый синтаксис."

# 1) Вызов tool
formatted = text_formatter.invoke({"text": raw_text, "width": 50, "indent": 2})
print("Результат text_formatter:\n", formatted)
print()

# 2) Модель генерирует краткое резюме для пользователя
summary_prompt = f"""### Instruction:
Пользователь попросил отформатировать текст. Текст отформатирован. Кратко (1-2 предложения) сообщи пользователю, что сделано.

Отформатированный текст:
---
{formatted}
---

### Response:
"""
print("Ответ модели:")
print(generate(summary_prompt, max_new_tokens=100))

## 4. Опционально: системный промпт «когнитивный дизайнер»

Для объяснений в стиле когнитивного дизайнера — см. `prompt_cognitive_designer.md` или `../06_prompting_guide/promt.md`. Добавь его в `full_prompt` перед запросом пользователя.

In [ ]:
# Пример с системным промптом-когнитивным дизайнером
# Инструкция — в формате, который модель знает: внутри Instruction
user_q = "Объясни, что такое замыкание в JavaScript."
instruction = f"Стиль: объясняй через аналогии из жизни, начни с «Представь...». Вопрос: {user_q}"
prompt_cognitive = f"""### Instruction:
{instruction}

### Response:
"""

print("С когнитивным дизайнером:")
print(generate(prompt_cognitive, max_new_tokens=250))